### TODO
- [ ] Add more data
- [ ] Add more issues from other repositories
- [ ] Add closed issues from nextjs

In [47]:
import pandas as pd
import json
import mysql.connector
import requests as req
from dotenv import load_dotenv
from datetime import datetime
from datetime import timezone
import os
load_dotenv()  # take environment variables from .env.


True

In [20]:
GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
pd.set_option('display.max_columns', None)


In [3]:
cnx = mysql.connector.connect(user='root', password='Rvega2015',
                              host='127.0.0.1',
                              database='github')

In [ ]:
df = pd.read_json('json/language/all.json')
df.sort_values(by='stargazers_count', ascending=False, inplace=True)
df = df[df['open_issues'] > 1000]

df.head(1)

In [57]:
def add_developer(developer) -> None:
        cursor = cnx.cursor()
        query = "INSERT INTO developers (id, login, html_url, type) VALUES (%s,%s,%s,%s)"
        values = (developer['id'], developer['login'], developer['html_url'], developer['type'])
        try :
            cursor.execute(query, values)
            cnx.commit()

        except mysql.connector.Error as err:
            print(err)
            pass
        cursor.close()


In [60]:
def developers_to_db():
    developers = []
    rows = df.shape[0]
    for i in range(rows):
        row = df.iloc[i]
        developer = row['owner']
        developers.append(developer)
        add_developer(developer)

In [6]:
important_columns = ['id', 'full_name', 'owner', 'size', 'watchers_count', 'stargazers_count', 'description', 'html_url', 'name', 'language', 'forks_count', 'open_issues_count', 'created_at', 'updated_at']

def get_important_fields(row) -> list:
    important_fields = []
    for column in important_columns:
        if column in list(row.keys()):
            if (column == 'owner'):
                important_fields.append(row[column]['id'])
            else:
                important_fields.append(row[column])
    return important_fields
def repositories_to_db():
    cursor = cnx.cursor()
    rows = df.shape[0]
    for i in range(rows):
        row = df.iloc[i].to_dict()
        important_fields = get_important_fields(row)

        query = "INSERT INTO repositories (id, full_name, owner_id, size, watchers_count, stargazers_count, description, url, name, language, forks_count, open_issues_count, created_at, pushed_at) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        try :
            cursor.execute(query, important_fields)
            cnx.commit()

        except mysql.connector.Error as err:
            print(err)
            pass
    cursor.close()

In [55]:
def check_if_user_exist(user):
    cursor = cnx.cursor()
    query = "SELECT * FROM developers WHERE id = %s"
    values = (user['id'],)
    cursor.execute(query, values)
    result = cursor.fetchall()
    cursor.close()
    return len(result) > 0


In [63]:

def issues_to_db():
    cursor = cnx.cursor()
    query = "SELECT id, full_name FROM repositories"
    cursor.execute(query)
    repositories = cursor.fetchall()
    cursor.close()
    for repo in repositories:
        full_name = repo[1]
        repo_id = repo[0]
        r = req.get(
            f"https://api.github.com/repos/{full_name}/issues?state=open&per_page=1",
            headers={'Authorization': f'token {GITHUB_TOKEN}'}
        )
        issues = r.json()
        for (idx, issue) in enumerate(issues):
            cursor = cnx.cursor()
            created_at = datetime.fromisoformat(
                issue['created_at'][:-1]).astimezone(timezone.utc)
            closed_at = datetime.fromisoformat(
                issue['closed_at'][:-1]).astimezone(timezone.utc) if issue['closed_at'] else None

            user_exist = check_if_user_exist(issue['user'])
            if not user_exist:
                add_developer(issue['user'])

            query = "INSERT INTO issues (id, body, closed_at, comments, created_at, labels, milestone, number, state, title, url, repo_id, user_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
            values = (issue['id'], issue['body'], closed_at, issue['comments'], created_at, str(
                issue['labels']), issue['milestone'], issue['number'], issue['state'], issue['title'], issue['url'], repo_id, issue['user']['id'])
            try:
                cursor.execute(query, values)
                cnx.commit()
            except mysql.connector.Error as err:
                print(err)
                pass
            cursor.close()


issues_to_db()


1062 (23000): Duplicate entry '1442350478' for key 'issues.PRIMARY'
1062 (23000): Duplicate entry '1442819358' for key 'issues.PRIMARY'
1062 (23000): Duplicate entry '1442702053' for key 'issues.PRIMARY'
1062 (23000): Duplicate entry '1443064151' for key 'issues.PRIMARY'
1062 (23000): Duplicate entry '1442914190' for key 'issues.PRIMARY'
1062 (23000): Duplicate entry '1443000319' for key 'issues.PRIMARY'
1062 (23000): Duplicate entry '1443118290' for key 'issues.PRIMARY'
1062 (23000): Duplicate entry '1441401836' for key 'issues.PRIMARY'
1062 (23000): Duplicate entry '1437554391' for key 'issues.PRIMARY'
1062 (23000): Duplicate entry '1442910902' for key 'issues.PRIMARY'
1062 (23000): Duplicate entry '1442669990' for key 'issues.PRIMARY'
1062 (23000): Duplicate entry '1441758361' for key 'issues.PRIMARY'
1062 (23000): Duplicate entry '1433902840' for key 'issues.PRIMARY'
1062 (23000): Duplicate entry '1441851976' for key 'issues.PRIMARY'
1062 (23000): Duplicate entry '1443085498' for k